In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from functools import partial
from shapely.ops import transform
import pyproj
import math
from shapely.ops import cascaded_union
from sklearn.cluster import DBSCAN
import mplleaflet
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn import metrics
from matplotlib.colors import LinearSegmentedColormap, ListedColormap

In [ ]:
def load_data():
    #Loads the CSV files and appends them into a single DataFrame
    column_types = {'Accident_Index': np.string_, 'LSOA_of_Accident_Location': np.string_}
    data13 = pd.read_csv('/home/shreyas/Desktop/major2/data/DfTRoadSafety_Accidents_2013.csv', dtype=column_types)
    data14 = pd.read_csv('/home/shreyas/Desktop/major2/data/DfTRoadSafety_Accidents_2014.csv', dtype=column_types)
    data15 = pd.read_csv('/home/shreyas/Desktop/major2/data/DfTRoadSafety_Accidents_2015.csv', dtype=column_types)
    data16 = pd.read_csv('/home/shreyas/Desktop/major2/data/DftRoadSafety_Accidents_2016.csv', dtype=column_types)
    return data16.append(data15.append(data14.append(data13)))

In [ ]:

def buffer_in_meters(lng, lat, radius):
    proj_meters = pyproj.Proj(init='epsg:3857')
    proj_latlng = pyproj.Proj(init='epsg:4326')
    
    project_to_meters = partial(pyproj.transform, proj_latlng, proj_meters)
    project_to_latlng = partial(pyproj.transform, proj_meters, proj_latlng)
    
    pt_latlng = Point(lng, lat)
    pt_meters = transform(project_to_meters, pt_latlng)
    
    buffer_meters = pt_meters.buffer(radius)
    buffer_latlng = transform(project_to_latlng, buffer_meters)
    return buffer_latlng

In [ ]:
data = load_data()

In [ ]:
data.shape


In [ ]:
def concave_hull(points, k):
    pass

In [ ]:
data=data[pd.to_numeric(data['Latitude'], errors='coerce').notnull()]  
data=data[pd.to_numeric(data['Longitude'], errors='coerce').notnull()] 
data.Latitude.astype('float64')
data.Longitude.astype('float64')
data.describe()


In [ ]:
len(data.index)             #data.shape

In [ ]:
# Create the radian longitude and latitude columns


data['rad_lng'] = data['Longitude'] * math.pi / 180.0
data['rad_lat'] = data['Latitude'] * math.pi / 180.0

In [ ]:
eps_in_meters = 50.0
num_samples = 10
earth_perimeter = 40070000.0  # In meters
eps_in_radians = eps_in_meters / earth_perimeter * (2 * math.pi)


In [ ]:
data['cluster'] = DBSCAN(eps=eps_in_radians, min_samples=num_samples, metric='haversine').fit_predict(data[['rad_lat', 'rad_lng']])
fig = px.scatter(data[['rad_lat', 'rad_lng']], x="rad_lat", y="rad_lng")
fig.show() 

In [ ]:
print(data.columns)

In [ ]:
type(data)

In [ ]:
labels= data['cluster'].to_frame()['cluster'].to_numpy()   #Dataframe to numpy array
print(labels)
print(len(labels))
type(labels)

In [ ]:
#Print all the unique cluster number with no data points it includes          

#Then get the frequency count of the non-negative labels
counts = np.bincount(labels[labels>=0])

print (counts)
#print(len(counts))


In [ ]:
np.amax(counts) #Largest cluster size

In [ ]:
num_clusters=len(set(labels)) #no of clusters
print(len(set(labels)))

In [ ]:
unique_labels=set(labels)

In [ ]:
# get colors and plot all the points, color-coded by cluster (or gray if not in any cluster, aka noise)
fig, ax = plt.subplots()
colors = plt.cm.rainbow(np.linspace(0, 1, len(unique_labels)))
coords = data[["Latitude", "Longitude"]].values
for cluster_label, color in zip(unique_labels, colors):
    
    size = 100
    if cluster_label == -1: #make the noise (which is labeled -1) appear as smaller gray points
        color='ivory'
        size=0
    
    # plot the points that match the current cluster label
    x_coords = coords[labels==cluster_label][:,1]
    y_coords = coords[labels==cluster_label][:,0]
    ax.scatter(x=x_coords, y=y_coords, c=color, edgecolor='k', s=size, alpha=0.5)

ax.set_title('Number of clusters: {}'.format(num_clusters))
plt.show()

# MONTE CARLO

Reference: Significant DBSCAN towards Statistically Robust Clustering 
https://dl.acm.org/doi/pdf/10.1145/3340964.3340968

In [ ]:
max_size=[]
alpha=0.02     #Significance level     #choosen on the basis of hit n trial
M=1000         #No. of iterations

In [ ]:
for i in range(M):
    random_subset=data.sample(n=400000)  
    random_subset['cluster']=DBSCAN(eps=eps_in_radians, min_samples=num_samples, metric='haversine').fit_predict(random_subset[['rad_lat', 'rad_lng']])
    labels=random_subset['cluster'].to_frame()['cluster'].to_numpy()
    count=np.bincount(labels[labels>=0])
    print(len(set(labels))) #No. of cluster in every iterations
    maximum = np.amax(count)
    max_size.append(maximum)

In [ ]:
max_size.sort(reverse=True)
max_size

In [ ]:
Threshold = max_size[math.ceil(alpha*M)]
print(Threshold)

In [ ]:
idx=[]                #stores labels of all the clusters having size > Threshold
for i in range(len(counts)):
    if(counts[i]>Threshold):
        idx.append(i)
print(len(idx))


In [ ]:
newDF=pd.DataFrame() #creates a new dataframe that's empty
for i in range(len(idx)):
    select_cluster=data.loc[data['cluster']==idx[i]]     #Adding all those points which belong to cluster having size > Threshold
    #print (select_cluster)
    newDF=newDF.append(select_cluster,ignore_index=True) # ignoring index is optional

In [ ]:
print(newDF)

# Plotting all the clusters

In [ ]:
# Group the observations by cluster identifier
Groups=data.groupby('cluster')

In [ ]:
##Create the list of cluster blobs


cluster=list()
blob=list()
freq=list()

for cluster_id, points in Groups:
    if cluster_id >= 0:
        buffer_radius=eps_in_meters * 0.6
        buffers=[buffer_in_meters(lon,lat,buffer_radius) for lon, lat in zip(points['Longitude'], points['Latitude'])]
        blobs=cascaded_union(buffers)
        blob.append(blobs)
        cluster.append(cluster_id)
        freq.append(len(points))

In [ ]:
# Create the GeoDataFrame from the cluster numbers and blobs
data={'cluster':cluster,'polygon':blob,'count':freq}

cluster_gdf=gpd.GeoDataFrame(pd.DataFrame(data), geometry='polygon')
cluster_gdf.crs={'init': 'epsg:4326'}

img=cluster_gdf.geometry.plot(linewidth=2.0, color='red', edgecolor='red')

#clusters with red borders and colors
mplleaflet.show(fig=img.figure, tiles='cartodb_positron',path='DB_red.html')

In [ ]:
from random import randint
my_colors = []

for i in range(4000):
    my_colors.append('#%06X' % randint(0, 0xFFFFFF))#Generating colors array for mapping them to different clusters


In [ ]:
#Different clusters with different colors
cmap = LinearSegmentedColormap.from_list('my cmap', my_colors)
img1 = cluster_gdf.plot(column='cluster', cmap=cmap)
mplleaflet.show(fig=img1.figure,  tiles='cartodb_positron', path='DB_colored.html')

# Plotting the significant clusters

In [ ]:
# Group the observations by cluster identifier
Groups=newDF.groupby('cluster')

In [ ]:
##Create the list of cluster blobs


cluster=list()
blob=list()
freq=list()

for cluster_id, points in Groups:
    if cluster_id >= 0:
        buffer_radius=eps_in_meters * 0.6
        buffers=[buffer_in_meters(lon,lat,buffer_radius) for lon, lat in zip(points['Longitude'], points['Latitude'])]
        blobs=cascaded_union(buffers)
        blob.append(blobs)
        cluster.append(cluster_id)
        freq.append(len(points))

In [ ]:
# Create the GeoDataFrame from the cluster numbers and blobs
data={'cluster':cluster,'polygon':blob,'count':freq}

cluster_gdf=gpd.GeoDataFrame(pd.DataFrame(data), geometry='polygon')
cluster_gdf.crs={'init': 'epsg:4326'}

img2=cluster_gdf.geometry.plot(linewidth=2.0, color='red', edgecolor='red')

#Clusters with red borders and colors
mplleaflet.show(fig=img2.figure, tiles='cartodb_positron',path='DB_Significant_red.html')

In [ ]:
#Different clusters with different colors
cmap=LinearSegmentedColormap.from_list('my cmap', my_colors)
img3=cluster_gdf.plot(column='cluster', cmap=cmap)
mplleaflet.show(fig=img3.figure,tiles='cartodb_positron',path='DB_Significant_colored.html')